# Tugas Besar I
### IF4071 Pembelajaran Mesin
### *Algoritma Clustering*
--------------------------
Kelompok:

- Diki Ardian Wirasandi (13515092)
- Irfan Ariq (13515112)
- Pratamamia Agung Prihatmaja (13515142)

In [1]:
from sklearn import datasets
import math
import numpy as np
from scipy import stats
import time

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Agglomerative Clustering
------------------------------------------
*Agglomerative clustering* merupakan suatu teknik dalam melakukan *clustering* yang menggunakan pendekatan *hierarchical*. Ide utama dari teknik ini adalah dengan melakukan penggabungan dua buah *cluster* dengan jarak terdekat pada setiap iterasi hingga diperoleh banyak *cluster* sesuai yang dikehendaki.

Dalam notasi *pseudocode*, algoritma *agglomerative clustering* adalah sebagai berikut.

```
WHILE (length(clusters_now) > nb_target_cluster) DO
    pair_merged = get_shortest_distance_pair(clusters_now)
    merge_cluster(pair_merged)
    update_cluster(clusters_now)
```

Terdapat beberapa pendekatan dalam menentukan pasangan *cluster* dengan jarak terdekat. Pendekatan tersebut adalah sebagai berikut.

1. ***Complete linkage***. Dilakukan dengan menghitung jarak antara dua titik terjauh pada kedua *cluster*.
![complete linkage](img/complete.png "Complete linkage")

2. ***Single linkage***. Dilakukan dengan menghitung jarak antara dua titik terdekat pada kedua *cluster*.
![single linkage](img/single.png "Single linkage")

3. ***Average linkage***. Dilakukan dengan menghitung jarak rata-rata antara semua pasangan titik pada kedua cluster.
![average linkage](img/average.png "Average linkage")

4. ***Average-group linkage***. Dilakukan dengan menghitung jarak antara *centroid* dari kedua *cluster*.
![average group linkage](img/average_group.png "Average-group linkage")

Teknik penghitungan jarak juga beragam. Dalam implementasi ini, diterapkan jarak *manhattan*, *euclidean*, dan *cosine*.

# Class

In [2]:
import numpy as np
import math

class AgglomerativeClustering:
    '''
    Kelas untuk mengakomodasi nilai metode agglomerative clustering
    '''
    
    # Nilai default parameter
    n_clusters = 2
    linkage = 'complete'
    metrics = 'euclidean'
    
    available_metrics = ['euclidean', 'manhattan', 'cosine']
    available_linkage = ['complete', 'single', 'average_group', 'average']
    
    def __init__(self, n_clusters=n_clusters, linkage=linkage, metrics=metrics):
        '''
        Inisiasi kelas. Parameter yang dibutuhkan untuk setiap kelas diinisiasi atau diisi dengan nilai default 
        '''
        if n_clusters <= 0:
            raise Exception('n_clusters must be higher than 0')
        if metrics not in self.available_metrics:
            raise Exception('No metrics \'' + str(metrics) + '\'. Available metrics '+ str(self.available_metrics))
        if linkage not in self.available_linkage:
            raise Exception('No linkage \'' + str(linkage) + '\'. Available linkage '+ str(self.available_linkage))
        self.metrics = metrics
        self.n_clusters = n_clusters
        self.linkage = linkage
        
    def __euclidean_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung euclidean distance di antara dua vector dengan panjang yang sama
        '''
        sum = 0
        if (len(data1) == len(data2)):
            for x1, x2 in zip(data1, data2):
                sum += (x1 - x2)**2
            dist = math.sqrt(sum)
            return dist
        else:
            raise Exception('Length doesn\'t match')

    def __manhattan_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung manhattan distance di antara dua vector dengan panjang yang sama
        '''
        sum = 0
        if (len(data1) == len(data2)):
            for x1, x2 in zip(data1, data2):
                sum += abs(x1 - x2)
            return sum
        else:
            raise Exception('Length doesn\'t match')
            
    def __cosine_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung cosine distance di antara dua vector dengan panjang yang sama
        '''
        return np.dot(data1, data2) / (np.linalg.norm(data1) * np.linalg.norm(data2))

    def __get_distance(self, data1, data2, metrics):
        '''
        Fungsi untuk menghitung jarak dua vector dengan metric pengukuran jarak yang telah ditentukan
        '''
        if (metrics == 'euclidean'):
            dist = self.__euclidean_distance(data1, data2)
        elif (metrics == 'manhattan'):
            dist = self.__manhattan_distance(data1, data2)
        elif (metrics == 'cosine'):
            dist = self.__cosine_distance(data1, data2)
        else:
            raise Exception('Metrics not defined')
        return dist
    
    def __complete_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan complete linkage
        '''
        max_dist = 0
        for v1 in cluster1:
            for v2 in cluster2:
                if (max_dist < dist_matrix[v1][v2]):
                    max_dist = dist_matrix[v1][v2]
        return max_dist

    def __single_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan single linkage
        '''
        min_dist = None
        for v1 in cluster1:
            for v2 in cluster2:
                if (min_dist is None) or (min_dist > dist_matrix[v1][v2]):
                    min_dist = dist_matrix[v1][v2]
        return min_dist

    def __average_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan average linkage
        '''
        sum_dist = 0
        count_dist = 0
        for v1 in cluster1:
            for v2 in cluster2:
                sum_dist += dist_matrix[v1][v2]
                count_dist += 1
        return float(sum_dist)/float(count_dist)

    def __group_average_linkage(self, cluster1, cluster2, data, distance):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan average group linkage
        '''
        data1 = [data[i] for i in cluster1]
        data2 = [data[i] for i in cluster2]

        avg1 = np.mean(data1, axis = 0)
        avg2 = np.mean(data2, axis = 0)

        return self.__get_distance(avg1, avg2, distance)
    
    def __calculate_distance_matrix(self, data, metrics):
        '''
        Fungsi untuk menghitung distance matrix untuk semua pasangan vector di dalam data
        '''
        dist_matrix = []
        for idx1, data1 in enumerate(data):
            curr_dist_matrix = []
            for idx2, data2 in enumerate(data):
                if (idx1 > idx2):
                    curr_dist_matrix.append(dist_matrix[idx2][idx1])
                else:
                    dist = self.__get_distance(data1, data2, metrics)
                    curr_dist_matrix.append(dist)
            dist_matrix.append(curr_dist_matrix)
        return dist_matrix
        
    def fit_predict(self, data):
        '''
        Fungsi untuk melakukan clustering secara agglomerative
        '''
        
        # preprocessing distance matrix
        if (self.linkage != 'average_group'):
            dist_matrix = self.__calculate_distance_matrix(data, self.metrics)
        # inisiasi cluster dengan satu elemen per cluster awal 
        clusters = [[i] for i, c in enumerate(data)]

        # melakukan iterasi hingga diperoleh jumlah cluster sesuai yang dikehendaki
        while(len(clusters) > self.n_clusters):
            min_dist = None
            merge_pair = (0, 0)
            
            # mencari cluster dengan jarak terdekat untuk di-merge
            for idx1, c1 in enumerate(clusters):
                for idx2, c2 in enumerate(clusters[(idx1 + 1) :]):
                    if (self.linkage == 'single'):
                        dist = self.__single_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'complete'):
                        dist = self.__complete_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'average'):
                        dist = self.__average_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'average_group'):
                        dist = self.__group_average_linkage(c1, c2, data, self.metrics)
                    else:
                        raise Exception('Linkage not defined')
                    if (min_dist == None) or (dist < min_dist):
                        min_dist = dist
                        merge_pair = (idx1, idx1 + 1 + idx2)
            
            # merge pasangan cluster dengan jarak terdekat
            result_cluster = []
            for idx, c in enumerate(clusters):
                if idx not in merge_pair:
                    result_cluster.append(c)

            result_cluster.append(clusters[merge_pair[0]] + clusters[merge_pair[1]])

            clusters = result_cluster

        # menampilkan hasil clustering
        result_per_item = np.full(len(data), 0)
        for idx, clust in enumerate(clusters):
            result_per_item[clust] = idx

        return result_per_item
    
    '''
    Getter untuk parameter
    '''
    def get_n_cluster(self):
        return self.n_clusters
    
    def get_linkage(self):
        return self.linkage
    
    def get_metrics(self):
        return self.metrics

## Experiments
Berikut adalah eksperimen yang dilakukan untuk melakukan *clustering* terhadap *dataset* iris. Digunakan *euclidean distance* dan berbagai variasi perhitungan *linkage* antara dua cluster untuk menerapkan metode *agglomerative clustering*.

In [3]:
def purity(cluster_pred, label):
    data_per_cluster = [[] for i in range(len(set(cluster_pred)))]
    for i,x in enumerate(cluster_pred):
        data_per_cluster[x].append(label[i])

    sum = 0
    for clust in data_per_cluster:
        sum += stats.mode(clust)[1][0]

    return sum/len(cluster_pred)

In [4]:
iris = datasets.load_iris()
data = iris.data
label = iris.target

#### Clustering dengan Complete Linkage

Pada percobaan ini, digunakan pendekatan *complete linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [5]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='complete', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.7341616153717041 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 0 2 0 2 0 2 0 0 0 0 2 0 2 0 0 2 0 2 0 2 2
 2 2 2 2 2 0 0 0 0 2 0 2 2 2 0 0 0 2 0 0 0 0 0 2 0 0 2 2 2 2 2 2 0 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Purity: 0.84


#### Clustering dengan Single Linkage

Pada percobaan ini, digunakan pendekatan *single linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [6]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='single', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.6534392833709717 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Purity: 0.68


#### Clustering dengan Average Linkage

Pada percobaan ini, digunakan pendekatan *complete linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [7]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='average', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 1.1986215114593506 s ----
Cluster prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 1 1 2 1 1 1 1
 1 1 2 2 1 1 1 1 2 1 2 1 2 1 1 2 2 1 1 1 1 1 2 1 1 1 1 2 1 1 1 2 1 1 1 2 1
 1 2]
Purity: 0.9066666666666666


#### Clustering dengan Average-group Linkage

Pada percobaan ini, digunakan pendekatan *average-group linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [8]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='average_group', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 43.254955530166626 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 0 2 0 0 0 0
 0 0 2 2 0 0 0 0 2 0 2 0 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 2 0 0 0 2 0 0 0 2 0
 0 2]
Purity: 0.9066666666666666


## Hasil dan Analisis

Dari keempat eksperimen di atas, dapat ditarik kesimpulan bahwa untuk dataset iris, metode *agglomerative* dapat diterapkan untuk melakukan *clustering*. Penggunaan teknik *average linkage* dan *average-group linkage* untuk menghitung jarak antara dua cluster menghasilkan *cluster* dengan *purity* tertinggi, yaitu 0.9067. Namun, *average-group linkage* membutuhkan waktu eksekusi yang lebih lama dibandingkan *average linkage*. Hal ini dikarenakan teknik *average-group linkage* tidak dapat menggunakan hasil *preprocessing distance matrix* sehingga perlu dilakukan komputasi ulang dalam menghitung jarak antara dua *cluster* .

# DBSCAN
----------------------------

DBSCAN merupakan salah satu algoritma clustering yang mengelompokkan data berdasarkan kedekatannya dengan data lain. Data yang dianggap dekat akan dijadikan satu kelompok. Data dianggap dekat dan disebut bertetangga dengan data lainnya apabila jaraknya kurang dari smaa dengan nilai tertentu. Nilai tersebut disebut `epsilon`.

Pada DBSCAN satu *instance* data dapat dikategorikan menjadi `core_point`, `border_point`, atau `noise_data`/outlier. Sebuah data disebut `core_point` apabila memiliki jumlah tetangganya lebih dari sama dengan nilai tertentu. Nilai tersebut disebut `min_pts`. Sebuah data dikatakan `border_point` apabila jumlah tetangganya tidak lebih dari `min_pts` namun memiliki tetangga yang merupakan `core_point`. Sedangkan `noise_data` atau outlier adalah data yang jumlah tetangganya tidak lebih dari `min_pts` dan tidak bertetangga dengan `border_point`. 

Setiap `core_point` dan tetangganya (baik itu `core_point` atau pun `border_point`) akan menjadi satu cluster yang sama. `noise_data` atau outlier merupakan data yang tidak memiliki cluster. 

Ilustrasi: 
![DBSCAN](img/dbscan.png "Ilustrasi DBSCAN")

Pada gambar diatas, titik yang berwarna merah merupakan `core_point`, titik yang berwarna kuning merupakan `border_point` dan titik yang berwarna biru merupakan `noise_data` atau outlier.

Perhitungan jarak yang dapat digunakan pada implemetasi DBSCAN ini ada dua macam yaitu jarak euclidean dan jarak manhtattan.

Berikut ini merupakan pseudocode dari DBSCAN:

```
DBSCAN(data, eps, min_pts):
    curr_label = 0
    for data_i in data:
        if data_i is core_point and not yet labelled:
            label = curr_label
            cluster(data_i) = label
            neighbour_stack = [neighbour(data_i)]
            while neighbour_stack is not empty:
                neighbour_data_i = neighbour_stack.pop
                if neighbour_data_i not yet labelled:
                    cluster(neighbour_data_i) = label
                    if neighbour_data_i is core point:
                        neighbour_stack,push(neighbour(neighbour_data_i))
           curr_label += 1
```

In [9]:
import numpy as np 
import math

class tes_DBSCAN:
    
    UNLABELLED_DATA = -1
    
    n_clusters = None
    result = None
        
    metrics = 'euclidean'    
    eps = 0.5
    min_pts = 5
    available_metrics = ['euclidean', 'manhattan']

    def __init__ (self, min_pts=min_pts, eps=eps, metrics=metrics):
        '''
        Inisiasi kelas dengan min_pts dan epsilon
        '''
        if eps <= 0:
            raise Exception('eps must be higher than 0')
        if min_pts <= 0:
            raise Exception('min_pts must be higher than 0')
        if metrics not in self.available_metrics:
            raise Exception('No metrics \'' + str(metrics) + '\'. Available metrics '+ str(self.available_metrics))
            
        self.min_pts = min_pts
        self.eps = eps
        self.metrics=metrics
        
    def __euclidean_distance(self, point_a, point_b):
        '''
        Fungsi untuk menghitung euclidean distance
        '''
        dist = 0
        for a, b in zip(point_a, point_b):
            dist += (a - b) * (a - b)
        return np.sqrt(dist)

    def __manhattan_distance(self, point_a, point_b):
        '''
        Fungsi untuk menghitung manhattan distance 
        '''
        dist = 0
        for a, b in zip(point_a, point_b):
            dist += abs(a - b)
        return dist
    
    def __distance(self, point_a, point_b, metrics=metrics):
        '''
        Fungsi untuk mencari jarak berdasarkan metricsnya
        '''
        if len(point_a) == len(point_b):
            if metrics == 'euclidean':
                return self.__euclidean_distance(point_a, point_b)
            if metrics == 'manhattan':
                return self.__manhattan_distance(point_a, point_b)
        else:
            raise Exception("feature length doesn't same")
    
    def fit_predict(self, data):
        '''
        Fungsi untuk mengelompolkkan data
        '''
        size_data = len(data)
        
        # generate all neighbours 
        neighbours = []
        for i in range(size_data):
            neighbour_i = []
            for j in range(size_data):
                if self.__distance(data[i], data[j], self.metrics) <= self.eps:
                    neighbour_i.append(j)
            neighbours.append(neighbour_i)
        
        # initialize label
        self.result = np.full((size_data), self.UNLABELLED_DATA)
        
        # giving label to data
        curr_label = 0
        for i in range(size_data):
            # if neighbours > min_pts (data_i is core points) and not yet labelled, then give label 
            if len(neighbours[i]) >= self.min_pts and self.result[i] == self.UNLABELLED_DATA: 
                label = curr_label
                # giving label to all neighbours
                neighbours_i = [i]
                while len(neighbours_i) > 0:
                    neigh_i = neighbours_i.pop()
                    # if not yet labelled then give label to data and the neighbours
                    if self.result[neigh_i] == self.UNLABELLED_DATA:
                        self.result[neigh_i] = label
                        # if neigh_i is core point, then give label to the neighbour
                        if len(neighbours[neigh_i]) >= self.min_pts:
                            neighbours_i += neighbours[neigh_i]
                curr_label += 1
        
        self.n_clusters = curr_label           
        return self.result
    
    def get_n_clusters(self):
        if self.n_clusters is None:
            print("No data")
        else:
            return self.n_clusters
    
    def get_epsilon(self):
        return self.eps

    def get_metrics(self):
        return self.metrics
    
    def get_min_pts(self):
        return self.min_pts
    

## Experiments

Berikut ini merupakan hasil eksperimen implementasi DBSCAN untuk clustering data iris menggunakan *euclidean disntance*.

In [10]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data
label = iris.target

In [11]:
from scipy import stats

def purity(cluster_pred, label):
    outlier = False
    size_data = len(cluster_pred)
    
    data_per_cluster = [[] for i in range(len(set(cluster_pred)))]
    for i,x in enumerate(cluster_pred):
        if x == -1:
            outlier = True
        data_per_cluster[x].append(label[i])

    sum = 0
    for clust in data_per_cluster:
        sum += stats.mode(clust)[1][0]
    
    if outlier:
        sum -= stats.mode(clust)[1][0]
        size_data -= len(clust)

    return sum/size_data

In [12]:
import time

dbscan = tes_DBSCAN(eps=0.5, min_pts=4, metrics='euclidean')

start = time.time()

pred = dbscan.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("---- Cluster: {} ----".format(dbscan.get_n_clusters()))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.2908132076263428 s ----
---- Cluster: 3 ----
Cluster prediction:
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0
  0  0  1  1  1  1  1  1  1  2  1  1  2  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  2  1  1
  1  1  2  1  1  1  1  1  1 -1 -1  1 -1 -1  1  1  1  1  1  1  1 -1 -1  1
  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1 -1 -1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1]
Purity: 0.708029197080292


In [13]:
dbscan = tes_DBSCAN(eps=1, min_pts=3, metrics='euclidean')

start = time.time()

pred = dbscan.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("---- Cluster: {} ----".format(dbscan.get_n_clusters()))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.24554896354675293 s ----
---- Cluster: 2 ----
Cluster prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
Purity: 0.6666666666666666


## Hasil Eksperimen

Dari kedua hasil eksperimen diatas, dapat dilihat bahwa DBSCAN mampu mengelompokkan data iris dalam waktu sekitar 0.2 - 0.3  detik. Namun kedua eksperimen menghasilkan nilai purity yang berbeda, hal ini dikarenakan perbedaan nilai `epsilon` dan `min_pts`. Dengan nilai `epsilon` 0.5 dan `min_pts` 4, DBSCAN menghasilkan 3 kluster dan beberapa data yang dianggap outlier. Sedangkan dengan nilai `epsilon` 1 dan `min_pts` 3, DBSCAN menghasilkan 2 kluster dan tanpa ada data yang dianggap outlier. Hal ini menunjukkan bahwa DBSCAN akan sangat bergantung terhadap kedua nilai tersebut dan ini merupakan tantangan dalam menggunakan DBSCAN.